In [31]:
import os
import numpy as np
import random
import pickle
import sys

from models_config.model_config_GSP import *
from utils.analysis import *
from utils.config import SAVE_DIR_DATA
from utils.simualte_data import simulate_data
from main_cross_validation import parrallel_run_A
from main_rep_kd import parrallel_run_B
from main_vanilla_kd import parrallel_run_vanilla_kd
from main_fitnet import parrallel_run_fitnet
from main_lsp import parrallel_run_lsp
from main_mskd import parrallel_run_mskd

# 1. Create Demo Dataset

In [32]:
print("Creating demo dataset...")
subjects = 10
nodes = 35
views = 6

mu_0 = 3
sigma_0 = 0.2
mu_1 = 2.8
sigma_1 = 0.3

adjs_0 = simulate_data(subjects, nodes, views, sigma_0, mu_0)
adjs_1 = simulate_data(subjects, nodes, views, sigma_1, mu_1)

adjs = adjs_0 + adjs_1

labels = [0] * subjects + [1] * subjects
if not os.path.exists(SAVE_DIR_DATA + "gender_data"):
    os.makedirs(SAVE_DIR_DATA + "gender_data") 

with open(SAVE_DIR_DATA+'gender_data/gender_data_edges', 'wb') as f:
    pickle.dump(adjs, f)

with open(SAVE_DIR_DATA+'gender_data/gender_data_labels', 'wb') as f:
    pickle.dump(labels, f)
print("Finished creating demo dataset")


Creating demo dataset...
Finished creating demo dataset


# 2. Train teacher model and student model

In [33]:
print("Pre-training teacher model and student model...")
run = 0
parrallel_run_A(run, "gender_data")
print("Finished Pre-training teacher model and student model")

Pre-training teacher model and student model...
/Users/lorenzostigliano/Documents/University/Imperial/TEST/thesis-imperial/model_data/gender_data/gcn/model_assessment/gcn
Run : 0
--------------------------------------------------------------------------
Main :  0 MainModel_3Fold_gender_data_gcn_run_0_fixed_init 3
CV :  0
Num training graphs:  14 ; Num test graphs:  6
Size of Training Set: 14
Size of Validation Set: 6
---------------------------------
Time taken for epoch 0: 0.02910780906677246
Train accuracy: 0.7142857142857143
Train loss: 0.5965239405632019
GCN Weights are saved:
Parameter containing:
tensor([[ 0.1638,  0.1405,  0.1534, -0.0574, -0.0579, -0.0187,  0.0495,  0.0684,
         -0.0149, -0.1094, -0.1276,  0.1624, -0.0456, -0.0225, -0.1036, -0.0841,
          0.0368,  0.0711, -0.0668, -0.0626,  0.1342, -0.0871, -0.1260, -0.0263,
          0.1606,  0.0365,  0.0244, -0.0213,  0.1055, -0.0839,  0.1143,  0.0402,
          0.0196,  0.0229, -0.1045]], requires_grad=True)
Validati

# 3. Train RepKD

In [6]:
print("Training RepKD...")
parrallel_run_B(run, "gender_data")
print("Finished RepKD...")

Training RepKD...
0
/Users/lorenzostigliano/Documents/University/Imperial/TEST/thesis-imperial/model_data/gender_data/gcn/model_assessment/gcn_student_lsp_ensamble_2
Run :  0
--------------------------------------------------------------------------
Main :  0 MainModel_3Fold_gender_data_gcn_student_lsp_ensamble_2_run_0_fixed_init 3
Num training graphs:  14 ; Num test graphs:  6
CV : 0
Size of Training Set: 14
Size of Validation Set: 6
---------------------------------
Time taken for epoch 0: 0.1640329360961914
Train total loss: 2.3177460772650584
Train teacher and student loss: 0.6893361295972552
Train within student loss for weights: 0.13345171617610113
GCN Weights are saved:
Parameter containing:
tensor([[ 0.0669,  0.1412,  0.1480,  0.1480,  0.0326, -0.1477,  0.0168, -0.1044,
         -0.1584,  0.1490,  0.1284, -0.1699,  0.0329, -0.0273, -0.0269, -0.0779,
          0.0637, -0.1000,  0.0612,  0.0855,  0.1204,  0.0633, -0.1664, -0.1083,
          0.0856,  0.0342, -0.1308, -0.0965,  0.1

## Getting the best model for RepKD

In [7]:
analysis_type="model_assessment"
dataset_split="val"
dataset="gender_data"
models_args=[
    gcn_student_lsp_ensamble_2_args,
    gcn_student_lsp_ensamble_3_args,
    gcn_student_lsp_ensamble_4_args,
    gcn_student_lsp_ensamble_5_args
]
views = [0]

#GET ALL OF THE MODEL BEST STUDENTS FOR ALL RUNS AND DIFFERENT PICKING CRITERIA FOR OUR MODEL 
all_model_metrics_mean, all_model_metrics_var, all_model_best_student = get_all_best_student(analysis_type, dataset_split, dataset, models_args, views)

gcn_student_lsp_ensamble_2
[[1.0, 0.0, 0], [0.9888888888888889, 0.0, 1], [1.0, 0.0, 0], [0.9888888888888889, 0.0, 1], [1.0, 0.0, 0]]
gcn_student_lsp_ensamble_3
[[1.0, 0.0, 0], [1.0, 0.0, 1], [1.0, 0.0, 2], [1.0, 0.0, 1], [1.0, 0.0, 2]]
gcn_student_lsp_ensamble_4
[[1.0, 0.0, 0], [1.0, 0.0, 1], [1.0, 0.0, 3], [1.0, 0.0, 1], [1.0, 0.0, 3]]
gcn_student_lsp_ensamble_5
[[1.0, 0.0, 0], [1.0, 0.0, 1], [1.0, 0.0, 3], [1.0, 0.0, 1], [1.0, 0.0, 3]]


In [8]:
df_rep, df_acuracy, _, index = final_student(all_model_metrics_mean, all_model_metrics_var, all_model_best_student, selection_method='weighted acc')

In [9]:
# Self-reproduciblity score of the best student in each ensemble
# x-axis: number of students in ensemble (2, 3, 4, 5)
# y-axis: view (data)
df_rep

,2,3,4,5
0,0.988889,1.0,1.0,1.0
1,0.988889,1.0,1.0,1.0


In [10]:
# Accuracy of perforemance of the best student in each ensemble
df_acuracy

,2,3,4,5
0,0.5,0.5,0.5,0.5
1,0.5,0.5,0.5,0.5


In [11]:
# Index of the best student in each ensemble
index

,2,3,4,5
0,1.0,1.0,1.0,1.0


# Comparison with baselines

In [12]:
print("Training using vanilla KD")
run = 0
parrallel_run_vanilla_kd(run, "gender_data")
print("Finished using vanilla KD")

Training using vanilla KD
0
/Users/lorenzostigliano/Documents/University/Imperial/TEST/thesis-imperial/model_data/gender_data/gcn/model_assessment/gcn_student
Run :  0
--------------------------------------------------------------------------
Main :  0 MainModel_3Fold_gender_data_gcn_student_run_0_fixed_init 3
Num training graphs:  14 ; Num test graphs:  6
CV : 0
MainModel_3Fold_gender_data_gcn_student_run_0_fixed_init_CV_0_view_0_with_teacher
Size of Training Set: 14
Size of Validation Set: 6
---------------------------------
Time taken for epoch 0: 0.03335380554199219
Train accuracy: 0.5
Train loss: 1.3755874463490076
Train Soft CE loss: 0.6982047217232841
Train CE loss: 0.677382726754461
GCN Weights are saved:
Parameter containing:
tensor([[ 0.0662,  0.1400,  0.1473,  0.1491,  0.0332, -0.1468,  0.0157, -0.1051,
         -0.1568,  0.1503,  0.1286, -0.1686,  0.0314, -0.0280, -0.0283, -0.0774,
          0.0654, -0.0998,  0.0620,  0.0849,  0.1213,  0.0633, -0.1676, -0.1090,
          0.

In [13]:
print("Training using fitnet")
run = 0
parrallel_run_fitnet(run, "gender_data")
print("Finished using fitnet")

Training using fitnet
0
/Users/lorenzostigliano/Documents/University/Imperial/TEST/thesis-imperial/model_data/gender_data/gcn/model_assessment/fitnet
Run :  0
--------------------------------------------------------------------------
Main :  0 MainModel_3Fold_gender_data_fitnet_run_0_fixed_init 3
Num training graphs:  14 ; Num test graphs:  6
CV : 0
MainModel_3Fold_gender_data_fitnet_run_0_fixed_init_CV_0_view_0_fitnet
Size of Training Set: 14
Size of Validation Set: 6
---------------------------------
Time taken for epoch 0: 0.037726402282714844
Train accuracy: 0.5
Train loss: 1.6798288651875086
Train Soft CE loss: 0.6987240995679583
Train CE loss: 0.6778817879302161
Train MSE loss: 1.516114822455815
GCN Weights are saved:
Parameter containing:
tensor([[ 0.0647,  0.1396,  0.1496,  0.1484,  0.0297, -0.1446,  0.0173, -0.0990,
         -0.1503,  0.1512,  0.1299, -0.1679,  0.0290, -0.0237, -0.0322, -0.0771,
          0.0693, -0.0968,  0.0626,  0.0799,  0.1238,  0.0639, -0.1708, -0.1029,
 

In [14]:
print("Training using lsp")
run = 0
parrallel_run_lsp(run, "gender_data")
print("Finished using lsp")

Training using lsp
0
/Users/lorenzostigliano/Documents/University/Imperial/TEST/thesis-imperial/model_data/gender_data/gcn/model_assessment/lsp
Run :  0
--------------------------------------------------------------------------
Main :  0 MainModel_3Fold_gender_data_lsp_run_0_fixed_init 3
Num training graphs:  14 ; Num test graphs:  6
CV : 0
MainModel_3Fold_gender_data_lsp_run_0_fixed_init_CV_0_view_0_lsp


Size of Training Set: 14
Size of Validation Set: 6
---------------------------------
Time taken for epoch 0: 0.14441609382629395
Train accuracy: 0.5
Train loss: 0.8366681103195462
Train KD loss: 0.07747760175594262
Train CE loss: 0.6817129062754768
GCN Weights are saved:
Parameter containing:
tensor([[ 0.0650,  0.1396,  0.1496,  0.1485,  0.0296, -0.1445,  0.0176, -0.0989,
         -0.1506,  0.1515,  0.1302, -0.1679,  0.0291, -0.0237, -0.0321, -0.0771,
          0.0692, -0.0968,  0.0628,  0.0799,  0.1238,  0.0640, -0.1707, -0.1030,
          0.0896,  0.0293, -0.1330, -0.1010,  0.1578,  0.1192, -0.0753, -0.0388,
         -0.1590, -0.0037, -0.1252]], requires_grad=True)
Validation accuracy: 0.3333333333333333
Validation Loss: 0.863009532292684
Average Memory Usage: 284.69140625 MB, Std: 0.0
Average Time: 0.14441609382629395, Std: 0.0
Num training graphs:  14 ; Num test graphs:  6
CV : 1
MainModel_3Fold_gender_data_lsp_run_0_fixed_init_CV_1_view_0_lsp
Size of Training Set: 14
Size of Valid

In [15]:
print("Training using mskd")
run = 0
parrallel_run_mskd(run, "gender_data")
print("Finished using mskd")

Training using mskd
0
/Users/lorenzostigliano/Documents/University/Imperial/TEST/thesis-imperial/model_data/gender_data/gcn/model_assessment/mskd
Run :  0
--------------------------------------------------------------------------
Main :  0 MainModel_3Fold_gender_data_mskd_run_0_fixed_init 3
Num training graphs:  14 ; Num test graphs:  6
CV : 0
MainModel_3Fold_gender_data_mskd_run_0_fixed_init_CV_0_view_0_mskd
Size of Training Set: 14
Size of Validation Set: 6
---------------------------------
Time taken for epoch 0: 0.227980375289917
Train accuracy: 0.5
Train loss: 3.5582360199519565
Train MSKD loss: 1.105498755268489e-06
Train CE loss: 0.6811308477606092
GCN Weights are saved:
Parameter containing:
tensor([[ 0.0641,  0.1390,  0.1486,  0.1482,  0.0295, -0.1449,  0.0170, -0.0992,
         -0.1506,  0.1507,  0.1291, -0.1684,  0.0286, -0.0244, -0.0326, -0.0775,
          0.0685, -0.0971,  0.0622,  0.0797,  0.1232,  0.0635, -0.1708, -0.1035,
          0.0890,  0.0289, -0.1334, -0.1012,  0.

# 4. Compare our method with the baselines

## 4.a. Get reproduciblity scores

In [17]:
#GET THE REPRODUCIBILITY SCORE FOR ALL MODELS 
mean_all_runs, mean_all_std = reproducibility_mulitple_runs_student_specific(
    dataset="gender_data", 
    views=[0], 
    models=[  
        "gcn", 
        "gcn_student",
        "gcn_student_teacher",
        "fitnet", 
        "lsp", 
        "mskd",
        "gcn_student_lsp_ensamble_4" 
    ], 
    CV=["3Fold", "5Fold", "10Fold"], 
    runs=[i for i in range(1)], 
    students=[
        [1]
    ], 
    model_args= [
        gcn_args,
        gcn_student_args,
        gcn_student_args, 
        gcn_fitnet_student_args, 
        gcn_lsp_student_args, 
        gcn_mskd_student_args,
        gcn_student_lsp_ensamble_4_args
    ])

In [24]:
import pandas as pd

reproducibility_frame = pd.DataFrame(mean_all_runs.T)

In [25]:
models=["gcn", "gcn_student", "gcn_student_teacher", "fitnet_gcn", "lsp_gcn", "mskd_gcn", "repKD"]
reproducibility_frame.columns = models
reproducibility_frame = reproducibility_frame*100

reproducibility_frame.T

,0,1
gcn,98.888889,98.888889
gcn_student,100.000000,100.000000
gcn_student_teacher,100.000000,100.000000
fitnet_gcn,99.166667,99.166667
lsp_gcn,99.166667,99.166667
mskd_gcn,99.166667,99.166667
repKD,100.000000,100.000000


In [26]:
reproducibility_frame_std = pd.DataFrame(mean_all_std.T)*100
models=["gcn", "gcn_student", "gcn_student_teacher", "fitnet_gcn", "lsp_gcn", "mskd_gcn", "repKD"]
reproducibility_frame_std.columns = models
reproducibility_frame_std.T

,0,1
gcn,0.0,0.0
gcn_student,0.0,0.0
gcn_student_teacher,0.0,0.0
fitnet_gcn,0.0,0.0
lsp_gcn,0.0,0.0
mskd_gcn,0.0,0.0
repKD,0.0,0.0


## 4.b. Get performance metrics

In [27]:
views=[0]
CV=["3Fold", "5Fold", "10Fold"]

runs = [i for i in range(1)]
dataset = "gender_data"
dataset_split="val"
analysis_type="model_assessment"
metrics = ["acc"]

final_model = ["gcn_student_lsp_ensamble_4"]
final_model_args = [gcn_student_lsp_ensamble_4_args]
best_student = [1]
baseline_models = ["gcn", "gcn_student", "gcn_student_teacher", "fitnet", "lsp", "mskd"]
baseline_models_args = [
    gcn_args,
    gcn_student_args,
    gcn_student_args, 
    gcn_fitnet_student_args, 
    gcn_lsp_student_args, 
    gcn_mskd_student_args
]

In [28]:
average_across_views_df, average_across_views_df_std_mean = performance_mulitple_runs_student_specific(final_model, final_model_args, best_student, baseline_models, baseline_models_args, views, CV, runs, dataset, dataset_split, analysis_type, metrics)

In [29]:
average_across_views_df = pd.DataFrame(average_across_views_df)
models=["gcn", "gcn_student", "gcn_student_teacher", "fitnet", "lsp", "mskd", "repKD"]

average_across_views_df.columns = models
average_across_views_df = average_across_views_df*100
average_across_views_df.T

,0,1
gcn,59.166667,59.166667
gcn_student,44.351852,44.351852
gcn_student_teacher,44.351852,44.351852
fitnet,44.351852,44.351852
lsp,42.685185,42.685185
mskd,41.018519,41.018519
repKD,50.000000,50.000000


In [30]:
average_across_views_df_std_mean = pd.DataFrame(average_across_views_df_std_mean)
models=["gcn", "gcn_student", "gcn_student_teacher", "fitnet", "lsp", "mskd", "repKD"]

average_across_views_df_std_mean.columns = models
average_across_views_df_std_mean.T*100

,0
gcn,0.0
gcn_student,0.0
gcn_student_teacher,0.0
fitnet,0.0
lsp,0.0
mskd,0.0
repKD,0.0
